In [1]:
import openai
from openai import OpenAI
import os
import time
import pandas as pd
from dotenv import load_dotenv 

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY') # For replication, either create an .env file and insert your OPENAI_API_KEY there or directly replace api key here
client = OpenAI()

In [15]:
# IF YOU CHOOSE TO RUN THIS SCRIPT INDIVIDUALLY, REPLACE "~/Desktop/Univ. of Chicago/AI Judges/" WITH THE ACTUAL FILE PATH WHERE THE REPLICATION PACKAGE IS STORED ON YOUR COMPUTER
user_file_path = os.path.expanduser("~/Desktop/Univ. of Chicago/AI Judges/")

In [16]:
# Directory where the text files are located
directory = os.path.join(user_file_path, "Replication Package", "Experiment Materials", "Input Materials")

# Dictionary containing lists of file paths
files_dict = {
    "instructions": [
        "Instructions, Briefs, & SOAF/Horvat_LC_Instructions.txt",
        "Instructions, Briefs, & SOAF/Horvat_NLC_Instructions.txt",
        "Instructions, Briefs, & SOAF/Vukovic_LC_Instructions.txt",
        "Instructions, Briefs, & SOAF/Vukovic_NLC_Instructions.txt"
    ],
    # Briefs and statement of agreed facts are in the same .txt files
    "briefs": [
        "Instructions, Briefs, & SOAF/Horvat_Sainovic.txt",
        "Instructions, Briefs, & SOAF/Horvat_Vasiljevic.txt",
        "Instructions, Briefs, & SOAF/Vukovic_Sainovic.txt",
        "Instructions, Briefs, & SOAF/Vukovic_Vasiljevic.txt"
    ],
    "statute": [
        "Cases & Statute/Statute.txt",
    ],
    "precedents": [
        "Cases & Statute/Appeal_Sainovic.txt", 
        "Cases & Statute/Appeal_Vasiljevic.txt"
    ], 
    "trials": [
        "Cases & Statute/Trial_Horvat_1.txt",
        "Cases & Statute/Trial_Horvat_2.txt",
        "Cases & Statute/Trial_Vukovic_1.txt",
        "Cases & Statute/Trial_Vukovic_2.txt"
    ]
}

# Function to read text from files
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Loop through the dictionary to read and store each text file with the original variable names
for category, file_list in files_dict.items():
    for file_name in file_list:
        file_path = os.path.join(directory, file_name)
        # Clean up the filename to create the variable name (removes spaces and extensions)
        key = os.path.splitext(os.path.basename(file_name))[0].replace(" ", "_")
        globals()[key] = read_text_file(file_path)

In [17]:
# Assigning model specifications (change as needed)
temperature = .7
model = 'gpt-4o-2024-05-13'

In [18]:
# Defining prompt and functions to summarize longer cases (precedent & trial judgment)
summarize_precedent_prompt = """Please summarize the following court case. Include an overview of the charges, the prosecution's and defense's main arguments, and the court's findings. Emphasize the reasoning behind the court’s decision and any notable legal interpretations. Ensure the summary captures the essence of the court’s conclusions and that it is thoroughly comprehensive.
Here is the case: """

summarize_trial1_prompt = """Please summarize the following court case. Include an overview of the charges, the prosecution's and defense's main arguments, and any other relevant legal information. Please note that because the case in its entirety exceeds GPT 4's context limit, you will only be provided with the first half of the case. Ensure the summary captures the essence of what the case is about and the legal issues it entails. Make sure your summary is thoroughly comprehensive.
Here is the first half of the trial judgment: """ 

def summarize_precedent(Precedent, seed):
        precedent_response = client.chat.completions.create(
            model=model,
            temperature=temperature,
            seed=seed,
            messages=[
            {"role": "system", "content": "You are an appeals judge in a pending case at the International Criminal Tribunal for the Former Yugoslavia (ICTY)."},
            {"role": "user", "content": summarize_precedent_prompt + Precedent}
        ]
        )
        return precedent_response.choices[0].message.content

def summarize_trial1(First_Half, seed):
        trial1_response = client.chat.completions.create(
            model=model,
            temperature=temperature,
            seed=seed,
            messages=[
            {"role": "system", "content": "You are an appeals judge in a pending case at the International Criminal Tribunal for the Former Yugoslavia (ICTY)."},
            {"role": "user", "content": summarize_trial1_prompt + First_Half}
        ]   
        )
        return trial1_response.choices[0].message.content

def summarize_trial2(trial1_summary, Second_Half, seed):
        summarize_trial2_prompt = f"""Please summarize the following court case. Note that this is not the full case; it is only the second half. This is due to the full case exceeding GPT 4's context limit. For reference, here is a summary of the first half of the case:{trial1_summary} 
        Now that you have a summary of the first half on hand, please summarize the remainder of the case. Include any relevant legal information. Emphasize the reasoning behind the court’s decision and any notable legal interpretations. Ensure the summary captures the essence of the court’s conclusions and that it is thoroughly comprehensive.
        Here is the second half of the trial judgment: """    
        trial2_response = client.chat.completions.create(
            model=model,
            temperature=temperature,
            seed=seed,
            messages=[
            {"role": "system", "content": "You are an appeals judge in a pending case at the International Criminal Tribunal for the Former Yugoslavia (ICTY)."},
            {"role": "user", "content": summarize_trial2_prompt + Second_Half}
        ]   
        )    
        return trial2_response.choices[0].message.content  

In [19]:
def simulate_judicial_decision(Instructions, Condition, Condition_Name, Statute, Precedent, First_Half, Second_Half, seed):
    # Determine LC_classification based on Instructions (See online appendix "A.1.3.: Instructions" of Spamann/Klöhn 2016; they insert a law comment ("LC") in half of the observations)
    # Similar to them, we observe no effect of the law comment and remove it in subsequent scripts (i.e. prompt engineering attempts)
    if Instructions.find("As an international tribunal, the procedure of the ICTY combines elements from common law and from civil law systems, some of which may seem unfamiliar to you.") != -1:
        LC_classification = "LC"
    else:
        LC_classification = "NLC"

    precedent_summary = summarize_precedent(Precedent, seed)
    time.sleep(10)
    trial1_summary = summarize_trial1(First_Half, seed)
    time.sleep(10)
    trial2_summary = summarize_trial2(trial1_summary, Second_Half, seed)
    time.sleep(10)

    final_input = f"""{Instructions}

    Here is the Statement of Agreed Facts, the Brief for the Defense, and the Brief for the Prosecution: 

    {Condition}

    Here is the Statute: 

    {Statute} 

    Here is the summary for the Precedent: 

    {precedent_summary} 

    Here is the summary for the first half of the Trial Judgment: 

    {trial1_summary}

    Here is the summary for the second half of the Trial Judgment: 

    {trial2_summary}
    
    Now that you have read all of the relevant information, please provide a decision on whether to affirm the lower court’s decision or not (indicate this by either saying “Affirm” or “Reverse”), as well as a brief 1-paragraph description of your rationale.
    """

    response = client.chat.completions.create(
        model=model,
        temperature=temperature,
        seed=seed,
        messages=[
            {"role": "system", "content": "You are an appeals judge in a pending case at the International Criminal Tribunal for the Former Yugoslavia (ICTY). Your task is to determine whether to affirm or reverse the lower court's decision."},
            {"role": "user", "content": final_input}
        ]  
    )
    final_decision = response.choices[0].message.content 

    output_directory = os.path.join(user_file_path, "Replication Package", "Results", "GPT Results")
    os.makedirs(output_directory, exist_ok=True)
    output_file_name = f"{Condition_Name}_{seed}_{LC_classification}.txt"
    path = os.path.join(output_directory, output_file_name)

    with open(path, "w") as file:
        file.write("Final Decision:\n") 
        file.write(final_decision)


In [ ]:
import random
random.seed(25)

seeds = [random.randint(0, 1000000) for _ in range(25)]

In [11]:
for seed in seeds:
    horvat_instructions = random.choice([Horvat_NLC_Instructions, Horvat_LC_Instructions])
    vukovic_instructions = random.choice([Vukovic_NLC_Instructions, Vukovic_LC_Instructions])

    simulate_judicial_decision(horvat_instructions, Horvat_Sainovic, "Horvat_Sainovic", Statute, Appeal_Sainovic, Trial_Horvat_1, Trial_Horvat_2, seed=seed)
    simulate_judicial_decision(horvat_instructions, Horvat_Vasiljevic, "Horvat_Vasiljevic", Statute, Appeal_Vasiljevic, Trial_Horvat_1, Trial_Horvat_2, seed=seed)
    simulate_judicial_decision(vukovic_instructions, Vukovic_Sainovic, "Vukovic_Sainovic", Statute, Appeal_Sainovic, Trial_Vukovic_1, Trial_Vukovic_2, seed=seed)
    simulate_judicial_decision(vukovic_instructions, Vukovic_Vasiljevic, "Vukovic_Vasiljevic", Statute, Appeal_Vasiljevic, Trial_Vukovic_1, Trial_Vukovic_2, seed=seed)